### Initial zipcode analysis
This notebook primarily used just to test *if* zipcode data is significant. It is. More complex use of zipcodes in other notebooks.

In [1]:
#import and preview data set
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('kc_house_data_cleaned.csv')
df['date'] = pd.to_datetime(df['date'])

In [22]:
dumzip = pd.get_dummies(df['zipcode'],prefix = 'zip',drop_first=True)
dumzip

,zip_98002,zip_98003,zip_98004,zip_98005,zip_98006,zip_98007,zip_98008,zip_98010,zip_98011,zip_98014,...,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21593,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
21594,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df['pricelog'] = np.log(df['price'])
df['price'] = (df['pricelog']-df['pricelog'].mean())/np.std(df['pricelog'])

In [56]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

X = dumzip
y = df['price']
incl = stepwise_selection(X,y,verbose=False)
len(incl)

D:\Alex\Anaconda3\envs\learn-env\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
D:\Alex\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.


60

In [55]:
temp = pd.concat([df['price'],dumzip],axis=1)
f = 'price~'+'+'.join([x for x in dumzip.columns])

model = ols(f,temp).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.528
Method:                 Least Squares   F-statistic:                     351.7
Date:                Tue, 02 Jun 2020   Prob (F-statistic):               0.00
Time:                        14:50:00   Log-Likelihood:                -22494.
No. Observations:               21597   AIC:                         4.513e+04
Df Residuals:                   21527   BIC:                         4.569e+04
Df Model:                          69                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.0542      0.036    -29.167      0.000      -1.125      -0.983
zip_98002     -0.2952      0.061     -4.868      0.000      -0.414      -0.176
zip_98003      0.0888      0.055      1.623      0.105      -0.018       0.196
zip_98004      2.8784      0.053     54.454      0.000       2.775       2.982
zip_98005      2.0266      0.064     31.597      0.000       1.901       2.152
zip_98006      2.0311      0.047     42.787      0.000       1.938       2.124
zip_98007      1.5065      0.068     22.089      0.000       1.373       1.640
zip_98008      1.4914      0.055     27.353      0.000       1.385       1.598
zip_98010      0.6808      0.078      8.772      0.000       0.529       0.833
zip_98011      1.1037      0.061     18.084      0.000       0.984       1.223
zip_98014      0.7690      0.071     10.757      0.000       0.629       0.909
zip_98019      0.8007      0.062     13.009      0.000       0.680       0.921
zip_98022      0.2117      0.058      3.669      0.000       0.099       0.325
zip_98023      0.0367      0.047      0.773      0.440      -0.056       0.130
zip_98024      1.2034      0.085     14.181      0.000       1.037       1.370
zip_98027      1.4423      0.050     29.132      0.000       1.345       1.539
zip_98028      0.9621      0.055     17.645      0.000       0.855       1.069
zip_98029      1.4928      0.053     28.335      0.000       1.390       1.596
zip_98030      0.1436      0.056      2.560      0.010       0.034       0.254
zip_98031      0.1734      0.055      3.149      0.002       0.065       0.281
zip_98032     -0.1803      0.071     -2.530      0.011      -0.320      -0.041
zip_98033      1.8859      0.049     38.512      0.000       1.790       1.982
zip_98034      1.1075      0.047     23.766      0.000       1.016       1.199
zip_98038      0.5088      0.046     11.085      0.000       0.419       0.599
zip_98039      3.7551      0.104     36.237      0.000       3.552       3.958
zip_98040      2.6777      0.055     49.061      0.000       2.571       2.785
zip_98042      0.1995      0.047      4.284      0.000       0.108       0.291
zip_98045      0.8022      0.059     13.657      0.000       0.687       0.917
zip_98052      1.5996      0.046     34.675      0.000       1.509       1.690
zip_98053      1.6423      0.050     33.001      0.000       1.545       1.740
zip_98055      0.1634      0.055      2.950      0.003       0.055       0.272
zip_98056      0.6929      0.050     13.947      0.000       0.595       0.790
zip_98058      0.4363      0.048      9.014      0.000       0.341       0.531
zip_98059      1.0093      0.048     20.981      0.000       0.915       1.104
zip_98065      1.2051      0.053     22.623      0.000       1.101       1.310
zip_98070      1.0144      0.073     13.885      0.000       0.871       1.158
zi

In [57]:
temp = pd.concat([df['price'],dumzip],axis=1)
f = 'price~'+'+'.join([x for x in incl])

model = ols(f,temp).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.529
Model:                            OLS   Adj. R-squared:                  0.528
Method:                 Least Squares   F-statistic:                     403.9
Date:                Tue, 02 Jun 2020   Prob (F-statistic):               0.00
Time:                        14:51:54   Log-Likelihood:                -22504.
No. Observations:               21597   AIC:                         4.513e+04
Df Residuals:                   21536   BIC:                         4.562e+04
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.9442      0.014    -67.423      0.000      -0.972      -0.917
zip_98004      2.7685      0.041     67.448      0.000       2.688       2.849
zip_98040      2.5677      0.043     59.386      0.000       2.483       2.652
zip_98112      2.3337      0.044     52.842      0.000       2.247       2.420
zip_98006      1.9211      0.034     56.807      0.000       1.855       1.987
zip_98023     -0.0733      0.034     -2.169      0.030      -0.140      -0.007
zip_98168     -0.3923      0.044     -8.883      0.000      -0.479      -0.306
zip_98039      3.6451      0.098     37.137      0.000       3.453       3.838
zip_98075      1.8708      0.039     48.135      0.000       1.795       1.947
zip_98033      1.7760      0.036     49.476      0.000       1.706       1.846
zip_98002     -0.4052      0.051     -7.996      0.000      -0.504      -0.306
zip_98199      1.7783      0.041     43.325      0.000       1.698       1.859
zip_98105      1.9143      0.048     40.297      0.000       1.821       2.007
zip_98052      1.4896      0.032     46.681      0.000       1.427       1.552
zip_98074      1.5779      0.036     44.344      0.000       1.508       1.648
zip_98119      1.9164      0.053     36.474      0.000       1.813       2.019
zip_98053      1.5323      0.037     41.444      0.000       1.460       1.605
zip_98005      1.9166      0.055     34.963      0.000       1.809       2.024
zip_98115      1.3806      0.032     43.537      0.000       1.318       1.443
zip_98103      1.2743      0.031     40.705      0.000       1.213       1.336
zip_98117      1.2652      0.032     39.054      0.000       1.202       1.329
zip_98109      1.9650      0.067     29.211      0.000       1.833       2.097
zip_98027      1.3323      0.037     36.376      0.000       1.261       1.404
zip_98029      1.3828      0.041     33.877      0.000       1.303       1.463
zip_98116      1.3616      0.040     33.765      0.000       1.283       1.441
zip_98102      1.9170      0.069     27.863      0.000       1.782       2.052
zip_98122      1.3907      0.043     32.568      0.000       1.307       1.474
zip_98077      1.5429      0.051     30.380      0.000       1.443       1.642
zip_98008      1.3814      0.043     32.000      0.000       1.297       1.466
zip_98177      1.4198      0.045     31.384      0.000       1.331       1.509
zip_98107      1.2838      0.044     28.924      0.000       1.197       1.371
zip_98144      1.1568      0.040     29.178      0.000       1.079       1.235
zip_98072      1.2183      0.044     27.771      0.000       1.132       1.304
zip_98034      0.9975      0.033     30.611      0.000       0.934       1.061
zip_98065      1.0951      0.042     26.343      0.000       1.014       1.177
zip_98007      1.3965      0.060     23.462      0.000       1.280       1.513
zi